In [1]:
import numpy as np
import pandas as pd
from ast import literal_eval

In [2]:
# Import BVS_result
# Metal charge by cell2mol or BVS (only cases that cell reconstruction finished successfully)
df_bvs = pd.read_csv("BVS_result_Mn.txt",delimiter="\t")

In [3]:
df_bvs # 2493 Complexes

,refcode,metal,charge_cell2mol,charge_BVS,dictionary,unique_indices
0,ABUBII,Mn,1,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",3
1,ABUKUG,Mn,3,7777,"{2: 0.455605, 3: 0.000186, 4: 0.96526, 5: 9999...",3
2,ABUMAN,Mn,2,2,"{2: 7.9e-05, 3: 1.125807, 4: 2.167596, 5: 9999...",3
3,ACACMN,Mn,3,7777,"{2: 2.371348, 3: 1.109403, 4: 0.314263, 5: 0.2...",1
4,ACAGUH,Mn,2,2,"{2: 7.5e-05, 3: 1.120006, 4: 2.1641, 5: 9999, ...",4
...,...,...,...,...,...,...
2488,ZUYBAY,Mn,2,2,"{2: 0.000596, 3: 1.04323, 4: 2.094933, 5: 9999...",2
2489,ZUYWEV,Mn,2,2,"{2: 6.6e-05, 3: 1.019689, 4: 2.064641, 5: 9999...",2
2490,ZZZKBM01,Mn,2,2,"{2: 0.376516, 3: 9999, 4: 9999, 5: 9999, 6: 99...",2
2491,ZZZKWO02,Mn,2,2,"{2: 0.011704, 3: 0.905934, 4: 1.97481, 5: 9999...",2


In [4]:
df_cif = pd.read_csv("cif_charge_Mn.txt",delimiter="\t") # charge extracted from cif files

In [5]:
df_cif # 2831 complexes

,refcode,metal,charge_cif
0,ABIZES,Mn,2
1,ABUBII,Mn,1
2,ABUKUG,Mn,3
3,ABUMAN,Mn,2
4,ACACMN,Mn,3
...,...,...,...
2826,ZUYWEV,Mn,2
2827,ZUZNOX,Mn,2
2828,ZZZKBM01,Mn,2
2829,ZZZKWO02,Mn,2


In [6]:
df = df_bvs.merge(df_cif, on='refcode', how='left')
df = df.drop(['metal_y', 'unique_indices'], axis=1)
df = df.rename(columns={'metal_x': 'metal'})
df['dictionary'] = df.dictionary.apply(lambda x: literal_eval(str(x)))
cols = ['refcode', 'metal', 'charge_cif', 'charge_cell2mol', 'charge_BVS', 'dictionary']
df = df[cols] 
df = df.rename(columns={'charge_cif': 'Cif', 'charge_cell2mol' :'cell2mol', 'charge_BVS' : 'BVS'})

In [7]:
thre_max = 0.5

In [8]:
df['bvs_min'] = df.dictionary.apply(lambda x: sorted(x, key=x.get)[0]
                                      if x[sorted(x, key=x.get)[0]] <= thre_max else False)
df['min'] = df.dictionary.apply(lambda x: x[sorted(x, key=x.get)[0]]
                                      if x[sorted(x, key=x.get)[0]] <= thre_max else False)
df['bvs_second_min'] = df.dictionary.apply(lambda x: sorted(x, key=x.get)[1]
                                      if x[sorted(x, key=x.get)[1]] <= thre_max else False)
df['second_min'] = df.dictionary.apply(lambda x: x[sorted(x, key=x.get)[1]]
                                      if x[sorted(x, key=x.get)[1]] <= thre_max else False)

In [9]:
condition = (lambda x: 
 x[sorted(x, key=x.get)[1]]- x[sorted(x, key=x.get)[0]]  # (second_min_delta - min_delta)
 if (x[sorted(x, key=x.get)[0]] <= thre_max and x[sorted(x, key=x.get)[1]] <= thre_max) 
 else False)
df['diff'] = df.dictionary.apply(condition)

In [10]:
df

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff
0,ABUBII,Mn,1,1,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False
1,ABUKUG,Mn,3,3,7777,"{2: 0.455605, 3: 0.000186, 4: 0.96526, 5: 9999...",3,0.000186,2,0.455605,0.455419
2,ABUMAN,Mn,2,2,2,"{2: 7.9e-05, 3: 1.125807, 4: 2.167596, 5: 9999...",2,0.000079,False,False,False
3,ACACMN,Mn,3,3,7777,"{2: 2.371348, 3: 1.109403, 4: 0.314263, 5: 0.2...",5,0.293312,4,0.314263,0.020951
4,ACAGUH,Mn,2,2,2,"{2: 7.5e-05, 3: 1.120006, 4: 2.1641, 5: 9999, ...",2,0.000075,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2488,ZUYBAY,Mn,2,2,2,"{2: 0.000596, 3: 1.04323, 4: 2.094933, 5: 9999...",2,0.000596,False,False,False
2489,ZUYWEV,Mn,2,2,2,"{2: 6.6e-05, 3: 1.019689, 4: 2.064641, 5: 9999...",2,0.000066,False,False,False
2490,ZZZKBM01,Mn,2,2,2,"{2: 0.376516, 3: 9999, 4: 9999, 5: 9999, 6: 99...",2,0.376516,False,False,False
2491,ZZZKWO02,Mn,2,2,2,"{2: 0.011704, 3: 0.905934, 4: 1.97481, 5: 9999...",2,0.011704,False,False,False


In [22]:
print("Total cases : {}".format(len(df)))
print("Cell2mol and Cif are same : {} cases / different {} cases".format(
    len(df[df['cell2mol'] != df['Cif']]), len(df[df['cell2mol'] == df['Cif']])))
print("bvs_min and Cif are same : {} cases / different {} cases".format(
    len(df[df['bvs_min'] != df['Cif']]), len(df[df['bvs_min'] == df['Cif']])))
df[['refcode', 'Cif', 'cell2mol', 'bvs_min', 'BVS']]

Total cases : 2493
Cell2mol and Cif are same : 525 cases / different 1968 cases
bvs_min and Cif are same : 543 cases / different 1950 cases


,refcode,Cif,cell2mol,bvs_min,BVS
0,ABUBII,1,1,False,9999
1,ABUKUG,3,3,3,7777
2,ABUMAN,2,2,2,2
3,ACACMN,3,3,5,7777
4,ACAGUH,2,2,2,2
...,...,...,...,...,...
2488,ZUYBAY,2,2,2,2
2489,ZUYWEV,2,2,2,2
2490,ZZZKBM01,2,2,2,2
2491,ZZZKWO02,2,2,2,2


In [55]:
df['bvs_min']

0       False
1           3
2           2
3           5
4           2
        ...  
2488        2
2489        2
2490        2
2491        2
2492        2
Name: bvs_min, Length: 2493, dtype: object

In [61]:
# df[(df['bvs_min'] != False) & (df['cell2mol'] == df['Cif'])]
df[(df['bvs_min'] == False) ]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
0,ABUBII,Mn,1,1,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
5,ACEDUH,Mn,1,1,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
7,ACTPMN,Mn,1,9999,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
27,AHICEA,Mn,2,1,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
29,AHOHEL,Mn,2,2,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,ZOVPAB,Mn,5,9999,8888,"{2: 0.849656, 3: 0.771541, 4: 0.750725, 5: 999...",False,False,False,False,False,False,False,False,False,False
2475,ZOWPAF,Mn,3,3,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
2479,ZUPDEV,Mn,2,2,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
2481,ZUSSUE,Mn,1,1,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False


# Diff

In [11]:
df['_0.1'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.1 and x['diff'] != False) else False), axis=1)
df['_0.2'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.2 and x['diff'] != False) else False), axis=1)
df['_0.3'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.3 and x['diff'] != False) else False), axis=1)
df['_0.4'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.4 and x['diff'] != False) else False), axis=1)
df['_0.5'] = df.apply((lambda x: x['bvs_min'] if (x['diff'] > 0.5 and x['diff'] != False) else False), axis=1)

In [12]:
df

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
0,ABUBII,Mn,1,1,9999,"{2: 9999, 3: 9999, 4: 9999, 5: 9999, 6: 9999, ...",False,False,False,False,False,False,False,False,False,False
1,ABUKUG,Mn,3,3,7777,"{2: 0.455605, 3: 0.000186, 4: 0.96526, 5: 9999...",3,0.000186,2,0.455605,0.455419,3,3,3,3,False
2,ABUMAN,Mn,2,2,2,"{2: 7.9e-05, 3: 1.125807, 4: 2.167596, 5: 9999...",2,0.000079,False,False,False,False,False,False,False,False
3,ACACMN,Mn,3,3,7777,"{2: 2.371348, 3: 1.109403, 4: 0.314263, 5: 0.2...",5,0.293312,4,0.314263,0.020951,False,False,False,False,False
4,ACAGUH,Mn,2,2,2,"{2: 7.5e-05, 3: 1.120006, 4: 2.1641, 5: 9999, ...",2,0.000075,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,ZUYBAY,Mn,2,2,2,"{2: 0.000596, 3: 1.04323, 4: 2.094933, 5: 9999...",2,0.000596,False,False,False,False,False,False,False,False
2489,ZUYWEV,Mn,2,2,2,"{2: 6.6e-05, 3: 1.019689, 4: 2.064641, 5: 9999...",2,0.000066,False,False,False,False,False,False,False,False
2490,ZZZKBM01,Mn,2,2,2,"{2: 0.376516, 3: 9999, 4: 9999, 5: 9999, 6: 99...",2,0.376516,False,False,False,False,False,False,False,False
2491,ZZZKWO02,Mn,2,2,2,"{2: 0.011704, 3: 0.905934, 4: 1.97481, 5: 9999...",2,0.011704,False,False,False,False,False,False,False,False


In [45]:
df[(df['BVS'] == 7777) & (df['diff'] > 0.1)]
df[(df['BVS'] == 7777) & (df['diff'] > 0.2)]
df[(df['BVS'] == 7777) & (df['diff'] > 0.3)]
df[(df['BVS'] == 7777) & (df['diff'] > 0.4)]

,refcode,metal,Cif,cell2mol,BVS,dictionary,bvs_min,min,bvs_second_min,second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
1,ABUKUG,Mn,3,3,7777,"{2: 0.455605, 3: 0.000186, 4: 0.96526, 5: 9999...",3,0.000186,2,0.455605,0.455419,3,3,3,3,False
80,AWOLOO,Mn,3,3,7777,"{2: 0.407432, 3: 0.00163, 4: 0.920535, 5: 9999...",3,0.00163,2,0.407432,0.405802,3,3,3,3,False
106,BAFTAG,Mn,2,2,7777,"{2: 0.001545, 3: 0.444221, 4: 1.531451, 5: 999...",2,0.001545,3,0.444221,0.442676,2,2,2,2,False
125,BEPTIA,Mn,3,3,7777,"{2: 0.49171, 3: 0.000108, 4: 0.884705, 5: 9999...",3,0.000108,2,0.49171,0.491602,3,3,3,3,False
144,BEZZEN,Mn,3,1,7777,"{2: 0.483892, 3: 0.000347, 4: 0.618814, 5: 999...",3,0.000347,2,0.483892,0.483545,3,3,3,3,False
175,BOLCUC,Mn,3,3,7777,"{2: 0.44707, 3: 1.2e-05, 4: 0.911055, 5: 9999,...",3,0.000012,2,0.44707,0.447058,3,3,3,3,False
176,BOLZAG,Mn,3,3,7777,"{2: 0.456953, 3: 0.000299, 4: 0.886284, 5: 999...",3,0.000299,2,0.456953,0.456654,3,3,3,3,False
364,DULVIR,Mn,4,9999,7777,"{2: 1.152816, 3: 0.493669, 4: 0.026778, 5: 999...",4,0.026778,3,0.493669,0.466891,4,4,4,4,False
365,DULVOX,Mn,4,9999,7777,"{2: 1.149763, 3: 0.468665, 4: 0.00013, 5: 9999...",4,0.00013,3,0.468665,0.468535,4,4,4,4,False
367,DUPWER,Mn,3,1,7777,"{2: 0.400018, 3: 7e-06, 4: 0.699802, 5: 9999, ...",3,0.000007,2,0.400018,0.400011,3,3,3,3,False


In [13]:
df_diff = df[df['diff'] != False]

In [16]:
df_diff = df_diff.drop(['metal', 'BVS', 'dictionary', 'min', 'second_min'], axis=1)

KeyError: "['metal' 'BVS' 'dictionary' 'min' 'second_min'] not found in axis"

In [17]:
df_diff

,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
1,ABUKUG,3,3,3,2,0.455419,3,3,3,3,False
3,ACACMN,3,3,5,4,0.020951,False,False,False,False,False
11,ADIPUB,3,1,3,2,0.006589,False,False,False,False,False
12,ADIQAI,3,1,3,2,0.007013,False,False,False,False,False
14,AETSMN,3,9999,3,2,0.32921,3,3,3,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2465,ZIWHES,4,9999,4,2,0.00407,False,False,False,False,False
2466,ZOFFAE,2,2,4,3,0.149684,4,False,False,False,False
2467,ZOFFEI,3,1,4,3,0.208904,4,4,False,False,False
2483,ZUTMIK,3,1,3,2,0.000021,False,False,False,False,False


In [42]:
for i in [0.1, 0.2, 0.3, 0.4, 0.5]:
    thre_diff = i
    # len(df_diff[df_diff[f'_{thre_diff}'] != False])
    print(f"The minimum diffrence between smallest delta and second smallest delat is {thre_diff}")
    print("The number of cases that bvs doesn't work in the above condition is {} out of {}\n".format(
        len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]), len(df_diff[df_diff[f'_{thre_diff}'] != False])))

The minimum diffrence between smallest delta and second smallest delat is 0.1
The number of cases that bvs doesn't work in the above condition is 13 out of 217

The minimum diffrence between smallest delta and second smallest delat is 0.2
The number of cases that bvs doesn't work in the above condition is 8 out of 173

The minimum diffrence between smallest delta and second smallest delat is 0.3
The number of cases that bvs doesn't work in the above condition is 5 out of 114

The minimum diffrence between smallest delta and second smallest delat is 0.4
The number of cases that bvs doesn't work in the above condition is 1 out of 58

The minimum diffrence between smallest delta and second smallest delat is 0.5
The number of cases that bvs doesn't work in the above condition is 0 out of 0



In [26]:
thre_diff = 0.1
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

13


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
181,BOVGID,2,1,3,2,0.378022,3,3,3,False,False
885,JEPMEX,4,9999,2,3,0.156996,2,False,False,False,False
1053,LEHSIB,2,2,3,2,0.350357,3,3,3,False,False
1186,MNICUR,2,9999,3,2,0.448291,3,3,3,3,False
1395,OMOGEC,4,9999,2,3,0.259465,2,2,False,False,False
1558,QIPPAG,3,9999,2,3,0.113703,2,False,False,False,False
1769,SIQMIO,5,9999,2,3,0.234226,2,2,False,False,False
2006,USIFOU,2,3,3,2,0.36927,3,3,3,False,False
2252,XELMIK,3,3,2,3,0.167167,2,False,False,False,False
2279,XILLIO,2,2,3,4,0.378837,3,3,3,False,False


In [27]:
thre_diff = 0.2
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

8


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
181,BOVGID,2,1,3,2,0.378022,3,3,3,False,False
1053,LEHSIB,2,2,3,2,0.350357,3,3,3,False,False
1186,MNICUR,2,9999,3,2,0.448291,3,3,3,3,False
1395,OMOGEC,4,9999,2,3,0.259465,2,2,False,False,False
1769,SIQMIO,5,9999,2,3,0.234226,2,2,False,False,False
2006,USIFOU,2,3,3,2,0.36927,3,3,3,False,False
2279,XILLIO,2,2,3,4,0.378837,3,3,3,False,False
2467,ZOFFEI,3,1,4,3,0.208904,4,4,False,False,False


In [28]:
thre_diff = 0.3
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

5


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
181,BOVGID,2,1,3,2,0.378022,3,3,3,False,False
1053,LEHSIB,2,2,3,2,0.350357,3,3,3,False,False
1186,MNICUR,2,9999,3,2,0.448291,3,3,3,3,False
2006,USIFOU,2,3,3,2,0.36927,3,3,3,False,False
2279,XILLIO,2,2,3,4,0.378837,3,3,3,False,False


In [29]:
thre_diff = 0.4
print(len(df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]))
df_diff[(df_diff[f'_{thre_diff}'] != df_diff['Cif'])& (df_diff[f'_{thre_diff}'] != False)]

1


,refcode,Cif,cell2mol,bvs_min,bvs_second_min,diff,_0.1,_0.2,_0.3,_0.4,_0.5
1186,MNICUR,2,9999,3,2,0.448291,3,3,3,3,False
